In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string as st
import re
import warnings
warnings.filterwarnings("ignore")
import spacy
en = spacy.load('en_core_web_sm')
import string
import os

In [115]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stopwords = []
wordnet_lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [116]:
import keras
from keras.preprocessing.text import Tokenizer

In [117]:
df = pd.read_csv('/content/train - train.csv')

In [118]:
df.head()

,text,aspect,label
0,can you check whether its cancelled completely?,cancelled,1
1,cannot rely on both milk delivery and grocery ...,Milk,0
2,"I get no notification, however the app is real...",notification,0
3,"Love this app, but would love it even more if ...",view,1
4,it does not let me load a clip on the scene,load,0


In [119]:
def lower_case(df):
  df['aspect']= df['aspect'].apply(lambda x: x.lower())
  df['text']= df['text'].apply(lambda x: x.lower())
  return df

In [120]:
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
df['aspect']= df['aspect'].apply(lambda x:remove_punctuation(x))

In [121]:
lower_case(df)

,text,aspect,label
0,can you check whether its cancelled completely?,cancelled,1
1,cannot rely on both milk delivery and grocery ...,milk,0
2,"i get no notification, however the app is real...",notification,0
3,"love this app, but would love it even more if ...",view,1
4,it does not let me load a clip on the scene,load,0
...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0
3996,unable to load money using wallets (phonepe/ol...,unable to load money using wallets,0
3997,"hi, i m doing for the first time ever and i no...",usage,0
3998,delivery is delayed or cancelled every time i ...,delivery,0


In [122]:
conj =['and','but','however']
punctuation_lst =['.','!',',',':',';','?']
conj= conj+punctuation_lst

In [123]:
def conj_based_split(text):
  lst=[]
  lst2 = []
  a=1
  for word in text.split():
    if word not in conj:
      lst.append(word)
    elif (word in conj):
      sent=' '.join(lst)
      lst2.append(sent)
      lst.clear()
    if (a==len(text.split())):
      if(len(lst)):
        sent=' '.join(lst)
        lst2.append(sent)
    a=a+1
  return lst2

In [124]:
def split(data):
  df['new_text']= df['text'].apply(lambda x:conj_based_split(x))

In [125]:
split(df)

In [126]:
df.head(20)

,text,aspect,label,new_text
0,can you check whether its cancelled completely?,cancelled,1,[can you check whether its cancelled completely?]
1,cannot rely on both milk delivery and grocery ...,milk,0,"[cannot rely on both milk delivery, grocery it..."
2,"i get no notification, however the app is real...",notification,0,"[i get no notification,, the app is really fine]"
3,"love this app, but would love it even more if ...",view,1,"[love this app,, would love it even more if ga..."
4,it does not let me load a clip on the scene,load,0,[it does not let me load a clip on the scene]
5,"i love notion as a tool, but the mobile just t...",tool,2,"[i love notion as a tool,, the mobile just tak..."
6,unlimited yearly at $216/year,yearly,1,[unlimited yearly at $216/year]
7,"some times missing to deliver, some times diff...",deliver,0,"[some times missing to deliver, some times dif..."
8,would love the ability to “un-delete” deleted ...,tasks,1,[would love the ability to “un-delete” deleted...
9,please add this basic feature,please,2,[please add this basic feature]


In [127]:
def find_aspect(df):
  result=[]
  for index in range(len(df)):
    #print(df['aspect'][index],df['new_text'][index])
    res = [i for i in df['new_text'][index] if df['aspect'][index] in i]
    result.append(res)
  #print(len(result))
  df['new_column'] = result
  return df

In [128]:
find_aspect(df)

,text,aspect,label,new_text,new_column
0,can you check whether its cancelled completely?,cancelled,1,[can you check whether its cancelled completely?],[can you check whether its cancelled completely?]
1,cannot rely on both milk delivery and grocery ...,milk,0,"[cannot rely on both milk delivery, grocery it...",[cannot rely on both milk delivery]
2,"i get no notification, however the app is real...",notification,0,"[i get no notification,, the app is really fine]","[i get no notification,]"
3,"love this app, but would love it even more if ...",view,1,"[love this app,, would love it even more if ga...",[calendar view were available on iphone!]
4,it does not let me load a clip on the scene,load,0,[it does not let me load a clip on the scene],[it does not let me load a clip on the scene]
...,...,...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0,[every time i try to edit a page or create a l...,[every time i try to edit a page or create a l...
3996,unable to load money using wallets (phonepe/ol...,unable to load money using wallets,0,[unable to load money using wallets (phonepe/o...,[unable to load money using wallets (phonepe/o...
3997,"hi, i m doing for the first time ever and i no...",usage,0,"[hi, i m doing for the first time ever, i not ...",[i not able to understand the usage of all the...
3998,delivery is delayed or cancelled every time i ...,delivery,0,[delivery is delayed or cancelled every time i...,[delivery is delayed or cancelled every time i...


In [129]:
df.head(20)

,text,aspect,label,new_text,new_column
0,can you check whether its cancelled completely?,cancelled,1,[can you check whether its cancelled completely?],[can you check whether its cancelled completely?]
1,cannot rely on both milk delivery and grocery ...,milk,0,"[cannot rely on both milk delivery, grocery it...",[cannot rely on both milk delivery]
2,"i get no notification, however the app is real...",notification,0,"[i get no notification,, the app is really fine]","[i get no notification,]"
3,"love this app, but would love it even more if ...",view,1,"[love this app,, would love it even more if ga...",[calendar view were available on iphone!]
4,it does not let me load a clip on the scene,load,0,[it does not let me load a clip on the scene],[it does not let me load a clip on the scene]
5,"i love notion as a tool, but the mobile just t...",tool,2,"[i love notion as a tool,, the mobile just tak...","[i love notion as a tool,]"
6,unlimited yearly at $216/year,yearly,1,[unlimited yearly at $216/year],[unlimited yearly at $216/year]
7,"some times missing to deliver, some times diff...",deliver,0,"[some times missing to deliver, some times dif...","[some times missing to deliver, some times dif..."
8,would love the ability to “un-delete” deleted ...,tasks,1,[would love the ability to “un-delete” deleted...,[would love the ability to “un-delete” deleted...
9,please add this basic feature,please,2,[please add this basic feature],[please add this basic feature]


In [130]:
def convertList(list1):  
    str = ''
    for i in list1: 
        str += i  
    return str  

In [131]:
df['new_column']= df['new_column'].apply(lambda x:convertList(x))

In [132]:
df=df.drop(columns=['new_text'])


In [133]:
df['text'] = df['new_column']


In [134]:
df=df.drop(columns=['new_column'])

In [135]:
df.drop(df.loc[df['label']==1].index, inplace=True)

In [136]:
df

,text,aspect,label
1,cannot rely on both milk delivery,milk,0
2,"i get no notification,",notification,0
4,it does not let me load a clip on the scene,load,0
5,"i love notion as a tool,",tool,2
7,"some times missing to deliver, some times diff...",deliver,0
...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0
3996,unable to load money using wallets (phonepe/ol...,unable to load money using wallets,0
3997,i not able to understand the usage of all the ...,usage,0
3998,delivery is delayed or cancelled every time i ...,delivery,0


In [137]:
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

In [138]:
def preprocess(data):
  def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in st.punctuation])
    return punctuationfree
  def tokenize(text):
    text = re.split('\s+' ,text)
    return [x.lower() for x in text]
  def remove_small_words(text):
    return [x for x in text if len(x) > 3 ]
  def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output
  def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
  def detokenizer(text):
    str1 = " " 
    return (str1.join(text))
  df['text']= df['text'].apply(lambda x:remove_punctuation(x))
  df['text']= df['text'].apply(lambda x: x.lower())
  df['aspect']= df['aspect'].apply(lambda x: x.lower())
  df['aspect']= df['aspect'].apply(lambda x:remove_punctuation(x))
  df['text'] = df['text'].apply(lambda x : tokenize(x))
  #df['text'] = df['text'].apply(lambda x : remove_small_words(x))
  #df['text'] = df['text'].apply(lambda x:remove_stopwords(x))
  #df['text']=df['text'].apply(lambda x:lemmatizer(x))
  df['text']=df['text'].apply(lambda x:detokenizer(x))
  return data

In [140]:
data = preprocess(df)

In [141]:
data

,text,aspect,label
1,cannot rely on both milk delivery,milk,0
2,i get no notification,notification,0
4,it does not let me load a clip on the scene,load,0
5,i love notion as a tool,tool,2
7,some times missing to deliver some times diffe...,deliver,0
...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0
3996,unable to load money using wallets phonepeola ...,unable to load money using wallets,0
3997,i not able to understand the usage of all the ...,usage,0
3998,delivery is delayed or cancelled every time i ...,delivery,0


In [142]:
#data[data.text.str.count(' ') > 0]

In [143]:
from sklearn.model_selection import train_test_split

In [144]:
x = data['text']
y = data['label']
x, x_test, y, y_test = train_test_split(x,y, stratify=y, test_size=0.25, random_state=42)

In [145]:
vec = CountVectorizer(stop_words='english')
x = vec.fit_transform(x).toarray()
x_test = vec.transform(x_test).toarray()

In [146]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x, y)

MultinomialNB()

In [147]:
model.score(x_test, y_test)

0.7872968980797637

In [148]:
model.predict(vec.transform(['this app could be better']))

array([0])

In [162]:
 ##################################

In [180]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [176]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(data['text'],data['label'],test_size=0.3)

In [181]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['text'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [183]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [185]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  79.67980295566502


In [218]:
###########################

In [219]:
def tokenize(text):
  text = re.split('\s+' ,text)
  return [x.lower() for x in text]

In [243]:
df['token_text'] = data['text'].apply(lambda x : tokenize(x))

In [244]:
df

,text,aspect,label,token_text
1,cannot rely on both milk delivery,milk,0,"[cannot, rely, on, both, milk, delivery]"
2,i get no notification,notification,0,"[i, get, no, notification]"
4,it does not let me load a clip on the scene,load,0,"[it, does, not, let, me, load, a, clip, on, th..."
5,i love notion as a tool,tool,2,"[i, love, notion, as, a, tool]"
7,some times missing to deliver some times diffe...,deliver,0,"[some, times, missing, to, deliver, some, time..."
...,...,...,...,...
3995,every time i try to edit a page or create a li...,tools,0,"[every, time, i, try, to, edit, a, page, or, c..."
3996,unable to load money using wallets phonepeola ...,unable to load money using wallets,0,"[unable, to, load, money, using, wallets, phon..."
3997,i not able to understand the usage of all the ...,usage,0,"[i, not, able, to, understand, the, usage, of,..."
3998,delivery is delayed or cancelled every time i ...,delivery,0,"[delivery, is, delayed, or, cancelled, every, ..."


In [226]:
X = df.token_text
Y = df['label']

In [227]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [228]:
max_features = 20000
#tokenizer = Tokenizer(num_words=max_features)
tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(list(X_train))
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)

In [231]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(y.values)
print(Y_train.values)

(2164,) (2164,)
(542,) (542,)
[0 2 2 ... 0 0 0]
[2 2 0 ... 0 2 2]


In [236]:
from keras.preprocessing.sequence import pad_sequences

In [237]:
maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [238]:
Y_train, Y_test = train_test_split(Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print(y.values)

(2164,) (2164,)
(542,) (542,)
[0 2 2 ... 0 0 0]


In [239]:
from keras.layers import Bidirectional, GlobalMaxPool1D

In [240]:
inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)
x = LSTM(200, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(120, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(60, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(4, activation="softmax")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_5 (Embedding)     (None, 100, 128)          2560000   
                                                                 
 lstm_layer (LSTM)           (None, 100, 200)          263200    
                                                                 
 global_max_pooling1d (Globa  (None, 200)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 120)               24120     
                                                             

In [241]:
model.fit(X_t, y_train , epochs=3, batch_size=32, validation_split=0.1)

Epoch 1/3
61/61 [==============================] - 25s 329ms/step - loss: 0.7909 - accuracy: 0.5511 - val_loss: 0.6526 - val_accuracy: 0.6636
Epoch 2/3
61/61 [==============================] - 20s 327ms/step - loss: 0.6825 - accuracy: 0.6061 - val_loss: 0.6351 - val_accuracy: 0.6636
Epoch 3/3
61/61 [==============================] - 22s 356ms/step - loss: 0.6530 - accuracy: 0.6225 - val_loss: 0.5709 - val_accuracy: 0.6636


In [242]:
scores = model.evaluate(X_te, y_test)
print("\n%s: %.4f%%" % (model.metrics_names[1], scores[1]*100))
y_pred = model.predict(X_te, batch_size=1024)
y_classes = y_pred.argmax(axis=-1)

17/17 [==============================] - 2s 136ms/step - loss: 0.5971 - accuracy: 0.6273

accuracy: 62.7306%


In [245]:
########################################

In [246]:
# Importing required libraries
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [247]:
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [248]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 128, 120)          60000     
                                                                 
 spatial_dropout1d (SpatialD  (None, 128, 120)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 176)               209088    
                                                                 
 dense_5 (Dense)             (None, 2)                 354       
                                                                 
Total params: 269,442
Trainable params: 269,442
Non-trainable params: 0
_________________________________________________________________
None


In [249]:
#Splitting the data into training and testing
y=pd.get_dummies(data['label'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [250]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
60/60 [==============================] - 44s 570ms/step - loss: 0.6481 - accuracy: 0.6235
Epoch 2/5
60/60 [==============================] - 37s 617ms/step - loss: 0.5024 - accuracy: 0.7719
Epoch 3/5
60/60 [==============================] - 35s 586ms/step - loss: 0.4189 - accuracy: 0.8136
Epoch 4/5
60/60 [==============================] - 35s 588ms/step - loss: 0.3896 - accuracy: 0.8268
Epoch 5/5
60/60 [==============================] - 35s 585ms/step - loss: 0.3702 - accuracy: 0.8400
